In [9]:
#imports
import requests
import pandas as pd
import os
from datetime import datetime
#from google.cloud import bigquery
from google.cloud import storage

In [10]:
CREDS = "C:/Users/tkkim/gcp_keys/capstone-team51-366963bafc54.json"
storage_client = storage.Client.from_service_account_json(json_credentials_path=CREDS,project='capstone-team51')
#client = bigquery.Client(project='capstone-team51')


In [16]:
bucket = storage_client.get_bucket('capstone-team51-data')

In [26]:
testblob = bucket.blob('raw_311/test.txt')

In [27]:
testblob.upload_from_string("Hello world")

In [21]:
blobs = bucket.list_blobs()


In [ ]:
testblob.upload_from_filename(path_to_file)


In [28]:
testblob.public_url

'https://storage.googleapis.com/capstone-team51-data/raw_311/test.txt'

In [22]:
blobs

In [23]:
for blob in blobs:
        print(blob.name)


/raw_311/test.txt
chatbot/
chatbot/ChicagoCrimeJan24.xlsx
consolidated/
consolidated/5_years_merged_with_weather_and_311and_policedistance.csv
logs/
raw_311/
raw_311/archive/
raw_311/archive/graffiti5_chunk_0.json
raw_311/archive/graffiti5_chunk_1.json
raw_311/archive/graffiti5_chunk_10.json
raw_311/archive/graffiti5_chunk_11.json
raw_311/archive/graffiti5_chunk_2.json
raw_311/archive/graffiti5_chunk_3.json
raw_311/archive/graffiti5_chunk_4.json
raw_311/archive/graffiti5_chunk_5.json
raw_311/archive/graffiti5_chunk_6.json
raw_311/archive/graffiti5_chunk_7.json
raw_311/archive/graffiti5_chunk_8.json
raw_311/archive/graffiti5_chunk_9.json
raw_crimes/
raw_crimes/archive/
raw_crimes/archive/crime5_chunk_0.json
raw_crimes/archive/crime5_chunk_1.json
raw_crimes/archive/crime5_chunk_2.json
raw_crimes/archive/crime_chunk_0.json
raw_environmental/
raw_environmental/archive/
raw_environmental/archive/envcomplaint5_chunk_0.json
supporting/
supporting/Police_Stations_20240120.csv
supporting/geojso

In [6]:
timestamp = datetime.now()
timestamp = timestamp.strftime("%Y%m%d")

In [ ]:
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)


In [ ]:
#primary data pull function

def get_data(name, db, maxrows, parameters):
    # some key info
    db_id = db
    limit = maxrows
    i = 0
    offset_counter = 0
    pickle = True
    data_name = name
    # dict for logging
    log_dict = {}

    # base url for the request
    baseurl = ("https://data.cityofchicago.org/resource/"
            f"{db_id}.json?")

    while pickle:
        #set our params
        # note that you have to URL econde 
        params = parameters + (
            f"&$limit={limit}"
            f"&$offset={offset_counter}"
            #f"&$$app_token={token}"
        )
        
        # make the request 
        data_req = requests.get(baseurl+params)

        # convert to json
        data_json = data_req.json()

        # make it a df
        df = pd.json_normalize(data_json)

        # write df
        
        
        # log
        minilog = {}
        minilog['records'] = df.shape[0]
        minilog['chunk'] = i
        minilog['status'] = data_req.status_code
        minilog['offset'] = offset_counter
        log_dict[f'run_{i}'] = minilog
        log_df = pd.DataFrame.from_dict(log_dict,orient='index')
        
        if df.shape[0] == 0:
            break
        else:
            df.to_json(f'{desti_folder}/{timestamp}_{data_name}_chunk_{i}.json',orient='records', lines=True)
            log_df.to_csv(f'{desti_folder}/{timestamp}_{data_name}_log.csv')

        # increment the chunk count
        print(i)
        i += 1

        # increment the offset
        offset_counter += limit

        # check if we need to end the loop
        if df.shape[0] == 0:
            pickle = False
        elif data_req.status_code == 200:
            pickle = True
        else: 
            pickle = False
            print(data_req.status_code)

    return df